#### Versions:
* v9: ColorJitter transformation added [0.896]
* v10: Changed the dataset to [this one](https://www.kaggle.com/shonenkov/melanoma-merged-external-data-512x512-jpeg) with external data. [0.894]
* v11: Switched to [another dataset](https://www.kaggle.com/nroman/melanoma-external-malignant-256/) which I've created by myself. Also switched from StratifiedKFold to GroupKFold [0.916]
* v12: Switched to efficientnet-b1 [0.919]
* v13: Using meta featues: sex and age [0.918]
* v14: anatom_site_general_challenge meta feature added as one-hot encoded matrix [0.923]
* v16: Fixed OOF - now it contains only data from original training dataset, without extarnal data. Also switched back to StratifiedKFold. Added DrawHair augmentation. [0.909]
* v18: Too many things were changed at the same time. All experiments should have only one small change each, so it would be easy to understand how changes affect the result. Said that I rolled back everything, keeping only OOF fix, to make sure it work.
* v19: Added 'Hair' augmentation. OOF rework posponed untill the best time, since there is some bug in my code for it.

In [1]:
!pip install efficientnet_pytorch torchtoolbox

     |████████████████████████████████| 84 kB 2.2 MB/s eta 0:00:011
     |████████████████████████████████| 294 kB 29.2 MB/s eta 0:00:01
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=5e8b839260b47b630b855e87bbcf6245196cd21ae1107033bafbc5cab07c85cb
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import torch
import torchvision
from torchvision import models,transforms
import torch.nn.functional as F
import torch.nn as nn
from torchvision import transforms
from torch.utils.data.sampler import RandomSampler, SequentialSampler

from torch.utils.data import Dataset, DataLoader, Subset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold, GroupKFold
import pandas as pd
import numpy as np
import gc
import os
import cv2
import time
import datetime
import warnings
import random
import matplotlib.pyplot as plt
import seaborn as sns
from efficientnet_pytorch import EfficientNet

from sklearn.model_selection import train_test_split

%matplotlib inline

In [3]:
torch.cuda.empty_cache()


In [4]:
# At least fixing some random seeds. 
# It is still impossible to make results 100% reproducible when using GPU
warnings.simplefilter('ignore')
torch.manual_seed(47)
np.random.seed(47)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
class MelanomaDataset(Dataset):
    def __init__(self, df: pd.DataFrame, train: bool = True, transforms = None, meta_features = None):
        """
        Class initialization
        Args:
            df (pd.DataFrame): DataFrame with data description
            imfolder (str): folder with images
            train (bool): flag of whether a training dataset is being initialized or testing one
            transforms: image transformation method to be applied
            meta_features (list): list of features with meta information, such as sex and age
            
        """
        self.df = df
        self.path1 = '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1'
        self.path2 = '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_2'
        self.transforms = transforms
        self.train = train
        self.meta_features = meta_features
        
    def __getitem__(self, index):
        image_path = os.path.join(self.path1, self.df.iloc[index]['image_id'] + '.jpg')
        if os.path.exists(image_path):
            image = cv2.imread(image_path)
        else:
            # If the image is not in part 1, try to load it from part 2
            image_path = os.path.join(self.path2, self.df.iloc[index]['image_id'] + '.jpg')
            if os.path.exists(image_path):
                image = cv2.imread(image_path)
                
        meta = np.array(self.df.iloc[index][self.meta_features].values, dtype=np.float32)

        if self.transforms:
            x = self.transforms(image)
            
        if self.train:
            y = self.df.iloc[index]['target']
            return (x, meta), y
        else:
            return (x, meta)
    
    def __len__(self):
        return len(self.df)
    
    def _get_label(self, dataset, idx):
        return self.df.iloc[idx]['target']
    
    

In [7]:
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])


In [8]:
skf = GroupKFold(n_splits=3)

In [9]:
# df = pd.read_csv('/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv')
# df = pd.read_csv('/kaggle/input/meta-abcd/metadata_ABCD.csv')
df = pd.read_csv('/kaggle/input/metadata-fromcnn/metadata_fromCNN.csv')


# this will tell us how many images are associated with each lesion_id
df_undup = df.groupby('lesion_id').count()
# now we filter out lesion_id's that have only one image associated with it
df_undup = df_undup[df_undup['image_id'] == 1]
df_undup.reset_index(inplace=True)

def get_duplicates(x):
    unique_list = list(df_undup['lesion_id'])
    if x in unique_list:
        return 'unduplicated'
    else:
        return 'duplicated'

# create a new colum that is a copy of the lesion_id column
df['duplicates'] = df['lesion_id']
# apply the function to this new column
df['duplicates'] = df['duplicates'].apply(get_duplicates)

df_undup  = df[df['duplicates'] == 'unduplicated']

#now we create a test set using df_undup because we are sure that none of these images have augmented duplicates in the train set
y = df_undup['dx']
_, test_df_csv = train_test_split(df_undup, test_size=0.36325, random_state=101, stratify=y)
test_df_csv.shape

# This set will be df_original excluding all rows that are in the test set
# This function identifies if an image is part of the train or test set.
def get_test_rows(x):
    # create a list of all the lesion_id's in the val test
    test_list = list(test_df_csv['image_id'])
    if str(x) in test_list:
        return 'test'
    else:
        return 'train'

# identify train and test rows
# create a new colum that is a copy of the image_id column
df['train_or_test'] = df['image_id']
# apply the function to this new column
df['train_or_test'] = df['train_or_test'].apply(get_test_rows)
# filter out train rows
train_df_csv = df[df['train_or_test'] == 'train']

# Split the test into 50% test and 50% validation
# test_df, val_df = train_test_split(test_df_csv, test_size=0.5, random_state=101, stratify=test_df_csv['dx'])

train_df = train_df_csv
test_df = test_df_csv
print(len(train_df))
print(len(test_df))

8012
2003


In [10]:
train_df_csv.dx.value_counts()

nv       5101
mel      1030
bkl       939
bcc       450
akiec     272
vasc      119
df        101
Name: dx, dtype: int64

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_df['target'] = le.fit_transform(train_df['dx'])
test_df['target'] = le.fit_transform(test_df['dx'])

In [12]:
train_df.target.value_counts()

5    5101
4    1030
2     939
1     450
0     272
6     119
3     101
Name: target, dtype: int64

In [13]:
# from collections import Counter
# from sklearn.utils import resample

# data_aug_rate = [14, 9, 4, 49, 4, 0, 39]

# for i in range(7):
#     if data_aug_rate[i] and i != train_df['target'].mode()[0]:
#         X_subset = train_df[train_df['target'] == i].drop(['target'], axis=1)
#         y_subset = train_df[train_df['target'] == i]['target']
#         X_subset_resampled, y_subset_resampled = resample(X_subset, y_subset, 
#                                                           replace=True, n_samples=data_aug_rate[i]*len(X_subset), 
#                                                           random_state=42)
#         train_df = pd.concat([train_df, pd.concat([X_subset_resampled, y_subset_resampled], axis=1)])
        
# train_df['target'].value_counts()


In [14]:
# data_aug_rate = [15,10,5,50,5,0,40]
# for i in range(7):
#     if data_aug_rate[i]:
#         train_df=train_df.append([train_df.loc[train_df['target'] == i,:]]*(data_aug_rate[i]-1), ignore_index=True)
# train_df['target'].value_counts()

In [15]:
train_df['sex'] = train_df['sex'].map({'male': 1, 'female': 0})
test_df['sex'] = test_df['sex'].map({'male': 1, 'female': 0})
train_df['sex'] = train_df['sex'].fillna(-1)
test_df['sex'] = test_df['sex'].fillna(-1)


In [16]:
imp_mean=(train_df["age"].sum())/(train_df["age"].count()-train_df["age"].isna().sum())
train_df['age']=train_df['age'].fillna(imp_mean)
train_df['age'].head()
imp_mean_test=(test_df["age"].sum())/(test_df["age"].count())
test_df['age']=test_df['age'].fillna(imp_mean_test)

In [17]:
train_df = pd.get_dummies(train_df, columns=['localization'],prefix='site')
test_df = pd.get_dummies(test_df, columns=['localization'],prefix='site')

# adding missing cols to testset
test_df = test_df.reindex(columns=train_df.columns, fill_value=0)

In [18]:
# meta_features = ['sex', 'age'] + [col for col in train_df.columns if 'site_' in col]
meta_features = ['age', 'sex'] + [col for col in train_df.columns if 'site_' in col]

In [19]:
cnn = list(range(0,256))
# abcd = list(range(0,6))

In [20]:
# cnn only
# meta_features = cnn

# meta_features = abcd             '''uncomment for cnn only'''
meta_features = meta_features + cnn

In [21]:
meta_features = [str(x) for x in meta_features]

In [22]:
len(meta_features)

273

In [23]:
#meta_features

In [24]:
test = MelanomaDataset(df=test_df,
                       train=False,
                       transforms=test_transform,
                        meta_features=meta_features)

In [25]:
class Net(nn.Module):
    def __init__(self, output_size, no_columns, b1=False, b4=False):
        super().__init__()
        self.no_columns = no_columns
        
        self.b1 = b1
        self.b4 = b4
        
        if self.b1:
            self.features = EfficientNet.from_pretrained('efficientnet-b1')
            self.classification = nn.Sequential(nn.Linear(1280 + 16, output_size))
        elif self.b4:
            self.features = EfficientNet.from_pretrained('efficientnet-b4')
            self.classification = nn.Sequential(nn.Linear(1792 + 16, output_size))
        
        
        # (CSV)
        self.csv = nn.Sequential(nn.Linear(self.no_columns, 32),
                                 nn.BatchNorm1d(32),
                                 nn.ReLU(),
                                 nn.Dropout(p=0.3),
                                 
                                 nn.Linear(32, 16),
                                 nn.BatchNorm1d(16),
                                 nn.ReLU(),
                                 nn.Dropout(p=0.2))
          
        
        
    def forward(self, image, csv_data, prints=False):    
        
        if prints: print('Input Image shape:', image.shape, '\n'+
                         'Input csv_data shape:', csv_data.shape)
        
        # IMAGE CNN
        image = self.features.extract_features(image)
        if prints: print('Features Image shape:', image.shape)

        if self.b1:
            image = F.avg_pool2d(image, image.size()[2:]).reshape(-1, 1280)
        elif self.b4:
            image = F.avg_pool2d(image, image.size()[2:]).reshape(-1, 1792)
     
        if prints: print('Image Reshaped shape:', image.shape)
            
        # CSV FNN
        csv_data = self.csv(csv_data)
        if prints: print('CSV Data:', csv_data.shape)
            
        # Concatenate
        image_csv_data = torch.cat((image, csv_data), dim=1)
        
#         image_csv_data = F.relu(image_csv_data)
        
        # CLASSIF
        out = self.classification(image_csv_data)
        if prints: print('Out shape:', out.shape)
        
        return out

In [26]:
epochs = 10 # Number of epochs to run
model_path = 'model.pth'  # Path and filename to save model to
es_patience = 3  # Early Stopping patience - for how many epochs with no improvements to wait
TTA = 3 # Test Time Augmentation 

oof = np.zeros((len(train_df), 7))  # Out Of Fold predictions


In [27]:
from tqdm import tqdm
import tqdm.notebook as tq
import torch.nn.functional as F
from sklearn.utils.class_weight import compute_class_weight

In [28]:
train_df.columns[train_df.isnull().any()]

Index([], dtype='object')

In [29]:
# Define Focal loss implementation
class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float,int)): self.alpha = torch.Tensor([alpha,1-alpha])
        if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
        self.reduction = reduction

    def forward(self, inputs, targets):
        CE_loss = F.cross_entropy(inputs, targets, reduction=self.reduction, weight=self.alpha)
        pt = torch.exp(-CE_loss)
        F_loss = ((1-pt)**self.gamma) * CE_loss
        if self.reduction == 'mean': return F_loss.mean()
        else: return F_loss.sum()

In [30]:
import gc

from sklearn.metrics import accuracy_score, precision_score,recall_score, f1_score

from sklearn.preprocessing import LabelBinarizer

from sklearn.metrics import confusion_matrix

In [31]:

# We stratify by target value, thus, according to sklearn StratifiedKFold documentation
# We can fill `X` with zeroes of corresponding length to use it as a placeholder
# since we only need `y` to stratify the data
# for fold, (train_idx, val_idx) in enumerate(skf.split(X=np.zeros(len(train_df)), y=train_df['target']), 1):
for fold, (train_idx, val_idx) in enumerate(skf.split(X=np.zeros(len(train_df)), y=train_df['target'], groups=train_df['lesion_id'].tolist()), 1):
    print('=' * 20, 'Fold', fold, '=' * 20)
    
    best_val = None  # Best validation score within this fold
    patience = es_patience  # Current patience counter
#     arch = EfficientNet.from_pretrained('efficientnet-b1')
#     arch = models.resnet50(pretrained=True)
#     model = Net(arch=arch, n_meta_features=len(meta_features))  # New model for each fold
    
    model = Net(output_size = 7 , no_columns=len(meta_features), b1=True)
    model = model.to(device)
    
    
    optim = torch.optim.Adam(model.parameters(), lr=0.0001)
    scheduler = ReduceLROnPlateau(optimizer=optim, mode='max', patience=1, verbose=True, factor=0.4)
    criterion = nn.CrossEntropyLoss()
    
    # Get the class weights for the training data
    #class_weights1 = torch.tensor(compute_class_weight('balanced', classes=np.unique(train_df.iloc[train_idx]['target']), y=train_df.iloc[train_idx]['target']), dtype=torch.float32)
    #class_weights = class_weights1.to(device)
    
    #giving more importance to mel
    
    # Define weight factors
    class_factor_interest = 2.0
    class_factor_minority = 1.0
    class_factor_majority = 0.5

    # Get class labels and weights
    class_labels = np.unique(train_df.iloc[train_idx]['target'])
    class_weights_all = compute_class_weight('balanced', classes=class_labels, y=train_df.iloc[train_idx]['target'])

    # Modify class weights to reflect weight factors
    class_weights = np.zeros_like(class_weights_all)
    for i, weight in enumerate(class_weights_all):
        if class_labels[i] == 4:  # class of interest
            class_weights[i] = weight * class_factor_interest
        elif weight == np.max(class_weights_all):  # majority class
            class_weights[i] = weight * class_factor_majority
        else:  # minority class
            class_weights[i] = weight * class_factor_minority

    # Convert class weights to PyTorch tensor
    class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)
    
    
    
    focal_loss = FocalLoss(gamma=2, alpha=class_weights)

    train = MelanomaDataset(df=train_df.iloc[train_idx].reset_index(drop=True), 
                            train=True, 
                            transforms=train_transform,
                            meta_features=meta_features)
    val = MelanomaDataset(df=train_df.iloc[val_idx].reset_index(drop=True), 
                            train=True, 
                            transforms=test_transform,
                            meta_features=meta_features)   

    train_loader = DataLoader(dataset=train, batch_size=16, shuffle=True, num_workers=0)
    val_loader = DataLoader(dataset=val, batch_size=8, shuffle=False, num_workers=0)
    test_loader = DataLoader(dataset=test, batch_size=8, shuffle=False, num_workers=0)
    
    for epoch in tqdm(range(epochs)):
        start_time = time.time()
        correct = 0
        epoch_loss = 0
        model.train()
        
        for x, y in tq.tqdm(train_loader):
            x[0] = torch.tensor(x[0], device=device, dtype=torch.float32)
            x[1] = torch.tensor(x[1], device=device, dtype=torch.float32)
            y = torch.tensor(y, device=device, dtype=torch.long)
            
            optim.zero_grad()
#             z = model(x)
            z = model(x[0],x[1])
            loss = focal_loss(z, y)
            loss.backward()
            optim.step()
            pred = torch.argmax(z, dim=1)  # get the index of the highest value in out
            correct += (pred.cpu() == y.cpu()).sum().item()  # tracking number of correctly predicted samples
            epoch_loss += loss.item()
        train_acc = correct / len(train_idx)

        model.eval()  # switch model to the evaluation mode
        val_preds = torch.zeros((len(val_idx), 7), dtype=torch.float32, device=device)
        with torch.no_grad():  # Do not calculate gradient since we are only predicting
            # Predicting on validation set
            for j, (x_val, y_val) in enumerate(val_loader):
                x_val[0] = torch.tensor(x_val[0], device=device, dtype=torch.float32)
                x_val[1] = torch.tensor(x_val[1], device=device, dtype=torch.float32)
                y_val = torch.tensor(y_val, device=device, dtype=torch.long)
#                 z_val = model(x_val)
                z_val = model(x_val[0],x_val[1])
                val_pred = torch.sigmoid(z_val)
                val_preds[j*x_val[0].shape[0]:j*x_val[0].shape[0] + x_val[0].shape[0]] = val_pred
            
            val_preds = torch.softmax(val_preds, dim=1) # use softmax to convert logits to probabilities
            val_acc = accuracy_score(train_df.iloc[val_idx]['target'].values, torch.argmax(val_preds.cpu(), dim=1))
            val_preds = val_preds.cpu().detach().numpy()
            val_preds = val_preds.reshape(-1, 7) # reshape val_preds to a 2D tensor
            
            val_roc = roc_auc_score(train_df.iloc[val_idx]['target'].values, val_preds, multi_class='ovr')
            
            print('Epoch {:03}: | Loss: {:.3f} | Train acc: {:.3f} | Val acc: {:.3f} | Val roc_auc: {:.3f} | Training time: {}'.format(
            epoch + 1, 
            epoch_loss/len(train_loader), 
            train_acc, 
            val_acc, 
            val_roc, 
            str(datetime.timedelta(seconds=time.time() - start_time))[:7]))
            
            scheduler.step(val_roc)
            # During the first iteration (first epoch) best validation is set to None
            if not best_val:
                best_val = val_roc  # So any validation roc_auc we have is the best one for now
                torch.save(model, model_path)  # Saving the model
                continue
                
            if val_roc >= best_val:
                best_val = val_roc
                patience = es_patience  # Resetting patience since we have new best validation accuracy
                torch.save(model, model_path)  # Saving current best model
            else:
                patience -= 1
                if patience == 0:
                    print('Early stopping. Best Val roc_auc: {:.3f}'.format(best_val))
                    break
                    
    gc.collect()
    torch.cuda.empty_cache()
    
    #Testing the fold results
    model = torch.load('/kaggle/working/model.pth')
    model.eval()  # switch model to the evaluation mode
    preds = torch.zeros((len(test), 7), dtype=torch.float32, device=device)
    with torch.no_grad():

        for i, x_test in enumerate(test_loader):
            x_test[0] = torch.tensor(x_test[0], device=device, dtype=torch.float32)
            x_test[1] = torch.tensor(x_test[1], device=device, dtype=torch.float32)
    #         z_test = model(x_test)
            z_test = model(x_test[0],x_test[1])
            z_test = torch.softmax(z_test, dim=1) # use softmax to convert logits to probabilities
            preds[i*x_test[0].shape[0]:i*x_test[0].shape[0] + x_test[0].shape[0]] += z_test



    gc.collect()   
        
    # Convert the target data to one-hot encoded format
    lb = LabelBinarizer()
    test_labels = lb.fit_transform(test_df['target'])

    # Get predictions for the test set
    test_preds = preds.cpu()

    # Convert the predictions to one-hot encoded format if needed
    if len(test_preds.shape) > 1 and test_preds.shape[1] > 1:
        test_preds = np.argmax(test_preds, axis=1)
        test_preds = lb.transform(test_preds)

    # Calculate metrics
    acc = accuracy_score(test_labels, test_preds)
    prec = precision_score(test_labels, test_preds, average='macro')
    rec = recall_score(test_labels, test_preds, average='macro')
    f1 = f1_score(test_labels, test_preds, average='macro')
    roc = roc_auc_score(test_labels, test_preds, multi_class='ovr')
    
    print("acc ", acc)
    print("prec ",prec)
    print("rec ",rec)
    print("f1 ",f1)
    print("roc ",roc)

    conf_matrix = confusion_matrix(test_labels.argmax(axis=1), test_preds.argmax(axis=1))
    
    # Extract the row and column corresponding to class 4: mel
    class_idx = 4
    fn = conf_matrix[class_idx, :].sum() - conf_matrix[class_idx, class_idx]

    # Calculate the percentage of instances from class 4 that were classified as another class
    fn_percent = (fn / conf_matrix[class_idx, :].sum()) * 100

    # Print the results
    print(f"False Negative Rate for mel: {fn_percent:.2f}%")

                

==================== Fold 1 ====================


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b1-f1951068.pth" to /root/.cache/torch/checkpoints/efficientnet-b1-f1951068.pth



Loaded pretrained weights for efficientnet-b1


  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [07:42<1:09:26, 462.97s/it]

...

 90%|█████████ | 9/10 [52:49<05:51, 351.10s/it]

Epoch 009: | Loss: 0.023 | Train acc: 0.861 | Val acc: 0.793 | Val roc_auc: 0.936 | Training time: 0:05:50
Epoch     9: reducing learning rate of group 0 to 1.6000e-05.


 90%|█████████ | 9/10 [58:39<06:31, 391.09s/it]

Epoch 010: | Loss: 0.021 | Train acc: 0.878 | Val acc: 0.810 | Val roc_auc: 0.938 | Training time: 0:05:50
Early stopping. Best Val roc_auc: 0.938


acc  0.9026460309535697
prec  0.6608934576090318
rec  0.7905219352914641
f1  0.7147997197281858
roc  0.8848558329180335
False Negative Rate for mel: 38.55%


In [ ]:
test_loader = DataLoader(dataset=test, batch_size=8, shuffle=False,num_workers=0)

In [ ]:
test_df.dx.value_counts()

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
model = torch.load('/kaggle/working/model.pth')
model.eval()  # switch model to the evaluation mode
preds = torch.zeros((len(test), 7), dtype=torch.float32, device=device)
with torch.no_grad():

    for i, x_test in enumerate(test_loader):
        x_test[0] = torch.tensor(x_test[0], device=device, dtype=torch.float32)
        x_test[1] = torch.tensor(x_test[1], device=device, dtype=torch.float32)
#         z_test = model(x_test)
        z_test = model(x_test[0],x_test[1])
        z_test = torch.softmax(z_test, dim=1) # use softmax to convert logits to probabilities
        preds[i*x_test[0].shape[0]:i*x_test[0].shape[0] + x_test[0].shape[0]] += z_test



gc.collect()   
           


In [ ]:
import gc

from sklearn.metrics import accuracy_score, precision_score,recall_score, f1_score

from sklearn.preprocessing import LabelBinarizer

# Convert the target data to one-hot encoded format
lb = LabelBinarizer()
test_labels = lb.fit_transform(test_df['target'])

# Get predictions for the test set
test_preds = preds.cpu()

# Convert the predictions to one-hot encoded format if needed
if len(test_preds.shape) > 1 and test_preds.shape[1] > 1:
    test_preds = np.argmax(test_preds, axis=1)
    test_preds = lb.transform(test_preds)

# Calculate metrics
acc = accuracy_score(test_labels, test_preds)
prec = precision_score(test_labels, test_preds, average='macro')
rec = recall_score(test_labels, test_preds, average='macro')
f1 = f1_score(test_labels, test_preds, average='macro')
roc = roc_auc_score(test_labels, test_preds, multi_class='ovr')


In [ ]:
print("acc ", acc)
print("prec ",prec)
print("rec ",rec)
print("f1 ",f1)
print("roc ",roc)

In [ ]:
from sklearn.metrics import confusion_matrix

import seaborn as sns
import matplotlib.pyplot as plt

# Generate the confusion matrix
conf_matrix = confusion_matrix(test_labels.argmax(axis=1), test_preds.argmax(axis=1))

# Create a heatmap of the confusion matrix
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')

# Set the axis labels and title
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')

# Display the plot
plt.show()


In [ ]:
# Extract the row and column corresponding to class 4: mel
class_idx = 4
fn = conf_matrix[class_idx, :].sum() - conf_matrix[class_idx, class_idx]

# Calculate the percentage of instances from class 4 that were classified as another class
fn_percent = (fn / conf_matrix[class_idx, :].sum()) * 100

# Print the results
print(f"False Negative Rate for mel: {fn_percent:.2f}%")